__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x20_d00_mwparserfromhell.ipynb)__ (allerdings wird es einen Timeout geben, weil das Notebook viele Stunden lang läuft))

# mwparserfromhell
* erstellt die Dateien wiktionary_merkmal.pickle und wiktionary_merkmal_text.pickle in data_00
* parst dafür wiktionary.wikitext mit mwparserfromhell
* File wiktionary wird nicht neu gespeichert
* Laufzeit je nach Anzahl der Kerne viele Stunden

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 22:02:17


time: 647 ms


In [2]:
# alle anderen
try:
    import pandasklar as pak, bj_nlp, mwparserfromhell, swifter

except ImportError as e:
    !pip install pandasklar bj_nlp mwparserfromhell swifter
    import pandasklar as pak, bj_nlp, mwparserfromhell, swifter
    
grid       = pak.grid
check_mask = pak.check_mask      

time: 5.94 s


## Einstellungen

In [3]:

version = '_all'
#version = '_short'
speicher_freigeben = True    # sollen alte Ergebnisse gelöscht werden

# Load
verzeichnis_load                 = 'data_00'
wiktionary_filename              = verzeichnis_load + '/wiktionary'               + version + '.pickle'  
wiktionary_trash_filename        = verzeichnis_load + '/wiktionary_trash'         + version + '.pickle'

# Save
verzeichnis_save                 = 'data_00'
wiktionary_merkmal_filename      = verzeichnis_save + '/wiktionary_merkmal'      + version + '.pickle'  
wiktionary_merkmal_text_filename = verzeichnis_save + '/wiktionary_merkmal_text' + version + '.pickle'  

# Was debuggen?
suche_debug = ['entlassen','missraten']

time: 41.9 ms


## Vorbereiten

In [4]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

time: 78.1 ms


In [5]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

time: 51.3 ms


In [6]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt (Notebook x10_d00_wiktionary_de_parser)
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

time: 57.3 ms


In [7]:
# Weitere Imports 
from Steuertabellen.s01_Steuertabellen   import *

# mwparserfromhell eigene Ergänzungen 
from x24_process_wikicode import *
from x22_my_node          import *

time: 136 ms


In [8]:
# Visualisiert einen Wikicode
def see(wikicode):
    return wikicode._get_tree(wikicode,list(),True,0)    

time: 52.7 ms


## Laden

In [9]:
# Laden
wiktionary       = pak.load_pickle(wiktionary_filename)
wiktionary_trash = pak.load_pickle(wiktionary_trash_filename)

time: 11.3 s


In [10]:
print('Einträge:', wiktionary.shape[0])

Einträge: 837593
time: 39.8 ms


In [11]:
# wiktionary anschauen
mask = wiktionary.title.isin(suche_debug) 
r = wiktionary[mask].sort_values(['title','section_id2'])
grid(r)

2 rows


time: 326 ms


In [12]:
pak.analyse_cols(wiktionary)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,int64,np.int64,int64,np.int32,True,False,True,False,80.0 B,1,837593,0,0,837593,0.0,418796.00,418796.0,837592.0,3.507806e+11
1,section_id,int32,pd.Int32,Int32,,True,False,True,True,50.0 B,1,837593,0,0,837593,0.0,62209823.28,62312700.0,111831800.0,5.210651e+13
2,page_id,int32,pd.Int32,Int32,,True,False,True,True,50.0 B,1,802086,0,35507,837593,0.0,622098.23,623127.0,1118318.0,5.210651e+11
3,page_part,int8,pd.Int8,Int8,,True,False,True,True,20.0 B,1,7,0,837586,837593,0.0,0.04,0.0,6.0,3.748300e+04
4,section_id2,str,pd.string,string,,False,True,True,True,792.0 B,1,837593,0,0,837593,&c.,NaN,NaN,μm,NaN
5,title,str,pd.string,string,,False,True,True,True,790.0 B,1,802084,0,35509,837593,&c.,NaN,NaN,μm,NaN
6,lemma,str,pd.string,string,,False,True,True,True,786.0 B,1,159601,0,677992,837593,&c.,NaN,NaN,μm,NaN
7,inflected,bool_,bool,bool,,True,False,True,True,10.0 B,1,2,0,837591,837593,0.0,0.83,NaN,1.0,6.969110e+05
8,pos,dict,object,object,,False,True,False,True,2.2 KB,1,163,0,837430,837593,NaN,NaN,NaN,NaN,NaN
9,rhymes,list,object,object,,False,True,False,True,512.0 B,1,39529,527175,270889,837593,[a],NaN,NaN,"[ʏʃəs, yːʃəs]",NaN


time: 16 s


In [13]:
pak.memory_consumption(locals())

,name,rtype,size
0,wiktionary,"(DataFrame, Series)",3.0 GB
1,wiktionary_trash,"(DataFrame, Series)",16.1 MB
2,mask,"(Series, bool)",818.1 KB
3,translate_tag,"(DataFrame, Series)",37.1 KB
4,translate_tagZ,"(DataFrame, Series)",20.9 KB
5,r,"(DataFrame, Series)",20.7 KB
6,_,"(DataFrame, Series)",16.3 KB
7,_12,"(DataFrame, Series)",16.3 KB
8,_i3,"(str,)",1.4 KB
9,Stop,"(type,)",1.0 KB


time: 1.17 s


## Steuertabelle plan_merkmal

In [14]:
blab.help(plan_merkmal_erstellen)

<class 'IPython.core.display.Markdown'>


<span style="font-size:larger;">plan_merkmal_erstellen():</span>

* definiert in Steuertabellen/s01_Steuertabellen.py
* `merkmal`: Name des Merkmals
* `template`: Name des Mediawiki-Templates, das diese Information enthält
* `collect`: Extractionsmethode
   * L extrahiert nur Links 
   * T extrahiert komplette Text-Absätze
   * Ü extrahiert Übersetzungen
   * P extrahiert Template-Parameter (Substring reicht)
   * E checkt nur auf Existenz
* `plan`:
   * 1: die Merkmale, die in wiktionary_merkmal gespeichert werden
   * 2: die Merkmale, die in wiktionary_merkmal_text gespeichert werden   
   * 3: Reihenfolge und Liste aller Merkmale aus flexion / übersicht <br>
        sowie die Merkmale aus named_entities und lexeme_manuell
* `sort`: Irgendwas zum Sortieren der Merkmale
* `is_lex`: Ist das Merkmal ein Lexem?    

time: 59.1 ms


In [15]:
plan_merkmal = plan_merkmal_erstellen()
grid(plan_merkmal)

103 rows


time: 180 ms


## ausprobieren

In [16]:
# Eine Testzeile aus wiktionary (als Series)
mask = (wiktionary.section_id2 == 'Haus')
testzeile = wiktionary[mask].iloc[0]
wikicode = mwparserfromhell.parse(testzeile.wikitext)

# testzeile 
# testzeile.wikitext
# wikicode

time: 452 ms


In [17]:
# extract_parameters ausprobieren (funktioniert z.B. mit 'kein')
r = extract_parameters(testzeile, 'Pronomina-Tabelle', 'übersicht', wikicode=wikicode )
grid(r, color=None)

No rows
time: 57.2 ms


In [18]:
#r.node_debug = r.node_debug.astype('str')
pak.analyse_cols(r)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,,object,object,,False,True,True,True,0.0 B,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


time: 195 ms


In [19]:
# extract ausprobieren
df = extract(testzeile, 'Unterbegriffe', 'down', wikicode=wikicode)
grid(df)

412 rows


time: 126 ms


In [20]:
# extract_all ausprobieren
mask = plan_merkmal.plan == 2
plan = plan_merkmal[mask]
df = extract_all(testzeile, plan) 
df

,node_num,node_meta,node_link,node_text,node_kontext,node_debug,section_id,section_id2,merkmal
0,[1],,,zu einem bestimmten Zweck erbautes Gebäude,,,21700,Haus,def
1,[2],,,zum Wohnen dienendes und genutztes Gebäude,,,21700,Haus,def
2,[3],,,"aus mehreren Räumen bestehender, abgetrennter ...",,,21700,Haus,def
3,[4],umgangssprachlich,,umgangssprachlich Gesamtheit der Bewohner in ...,,,21700,Haus,def
4,[5],,,"Gesamtheit der Personen, die sich in einem bes...",,,21700,Haus,def
...,...,...,...,...,...,...,...,...,...
5,"[2, 3]",,,mein Haus ist meine Burg,,,21700,Haus,bsp_sp
6,"[2, 3, 7, 9]",,,ein Haus kann nicht zwei Hunde nähren,,,21700,Haus,bsp_sp
7,[3],,,"lässt du einen ins Haus kommen, er kommt dir b...",,,21700,Haus,bsp_sp
8,[3],,,"Nord, Ost, Süd, West, zu Haus ist’s am best",,,21700,Haus,bsp_sp


time: 1.47 s


## wiktionary_merkmal_text

In [21]:
# plan
mask = plan_merkmal.plan == 2
plan = plan_merkmal[mask]
plan

,merkmal,template,collect,plan,sort,is_lex
38,def,Bedeutungen,T,2,,False
39,herkunft,Herkunft,T,2,,False
40,bsp,Beispiele,T,2,,False
41,bsp_re,Redewendungen,T,2,,False
42,bsp_gw,Geflügelte Worte,T,2,,False
43,bsp_sp,Sprichwörter,T,2,,False


time: 70.7 ms


In [22]:
# all: 40 min (4 parallel) 
# 100K: 3 min
# apply erzeugt eine Series von DataFrames
# tolist macht daraus eine Liste
# und concat macht aus der Liste von DataFrames ein Gesamt-DataFrame
wiktionary_merkmal_text = pd.concat(  wiktionary.swifter.apply(extract_all, plan=plan, axis=1).tolist()  )

Pandas Apply:   0%|          | 0/837593 [00:00<?, ?it/s]

Executing shutdown due to inactivity...


2023-02-16 23:02:58,096 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2023-02-16 23:02:58,153 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdown_server
    raise RuntimeError("Not 

2023-02-16 23:02:58,166 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdow

time: 3h 52min 47s


In [23]:
# nachbearbeiten
wiktionary_merkmal_text = pak.reset_index( wiktionary_merkmal_text )
wiktionary_merkmal_text = pak.move_cols(   wiktionary_merkmal_text, ['section_id','section_id2','merkmal'])
wiktionary_merkmal_text = pak.drop_cols(   wiktionary_merkmal_text, ['node_link','node_kontext','node_debug'])
wiktionary_merkmal_text = pak.rename_col(  wiktionary_merkmal_text, 'node_num',  'num')
wiktionary_merkmal_text = pak.rename_col(  wiktionary_merkmal_text, 'node_meta', 'meta')
wiktionary_merkmal_text = pak.rename_col(  wiktionary_merkmal_text, 'node_text', 'data')    

time: 1.21 s


## Kontrolle: wiktionary_merkmal_text

In [24]:
# Merkmale Häufigkeit
v = pak.analyse_freqs(wiktionary_merkmal_text, 'merkmal') 
v

,merkmal,merkmal_count,merkmal_percent,graph
0,bsp,355995,54.4,###########################
1,def,188700,28.8,##############
2,herkunft,103107,15.7,#######
3,bsp_re,6131,0.9,
4,bsp_sp,640,0.1,
5,bsp_gw,104,0.0,


time: 250 ms


In [25]:
# plan erfüllt?
geplant = set(plan.merkmal)
done    = set(v.merkmal)
nicht_erfüllt = geplant - done
print(nicht_erfüllt)
assert len(nicht_erfüllt) == 0

set()
time: 31.2 ms


In [26]:
# Sichtkontrolle
mask = wiktionary.section_id2.isin(suche_debug)
suche_section_ids = wiktionary[mask].section_id.tolist()
mask = wiktionary_merkmal_text.section_id.isin(suche_section_ids)
wiktionary_merkmal_text[mask]

,section_id,section_id2,merkmal,num,meta,data
83526,2270300,entlassen,def,[1],transitiv,transitiv jemandem erlauben zu gehen
83527,2270300,entlassen,def,[2],transitiv,transitiv jemandem die Arbeitsstelle kündigen
83528,2270300,entlassen,herkunft,[],etymologisch,etymologisch von mittelhochdeutsch entlāʒe...
83529,2270300,entlassen,bsp,[1],entlassen,Die Schüler wurden für heute entlassen.
83530,2270300,entlassen,bsp,[1],entlassen,Er wurde aus dem Gefängnis entlassen.
83531,2270300,entlassen,bsp,[1],entlassen,„Als er nach rund drei Monaten [Anmerkung: aus...
83532,2270300,entlassen,bsp,[2],entlassen,Sie wurde von der Firma entlassen.
83533,2270300,entlassen,bsp,[2],entlassen,„Der St. Gallener Polizeikommandant Paul Grüni...
300519,24283800,missraten,def,[1],,nicht gelingen
300520,24283800,missraten,bsp,[1],missraten,Dieser Kuchen ist vollkommen missraten.


time: 133 ms


## Schon mal speichern

In [27]:
# Speichern   
if wiktionary_merkmal_text_filename:
    pak.dump_pickle(wiktionary_merkmal_text, wiktionary_merkmal_text_filename)   

time: 5.22 s


In [28]:
# Speicher
if speicher_freigeben:
    try:
        del wiktionary_merkmal_text
    except:
        pass
pak.memory_consumption(locals())

,name,rtype,size
0,wiktionary,"(DataFrame, Series)",3.0 GB
1,wiktionary_trash,"(DataFrame, Series)",16.1 MB
2,mask,"(Series, bool)",639.5 KB
3,testzeile,"(Series, int32)",347.7 KB
4,df,"(DataFrame, Series)",91.3 KB
5,_20,"(DataFrame, Series)",91.3 KB
6,translate_tag,"(DataFrame, Series)",37.1 KB
7,plan_merkmal,"(DataFrame, Series)",26.9 KB
8,translate_tagZ,"(DataFrame, Series)",20.9 KB
9,_12,"(DataFrame, Series)",16.3 KB


time: 4.07 s


In [29]:
#pak.analyse_cols(wiktionary)

time: 30 ms


## wiktionary_merkmal

In [30]:
# plan
mask = plan_merkmal.plan == 1
plan = plan_merkmal[mask]
plan

,merkmal,template,collect,plan,sort,is_lex
0,noSteig,kSt.,E,1,,False
1,noSing,kSg.,E,1,,False
2,nurWortart,Wortart fehlt,P,1,,False
3,istName,Navigationsleiste Anthroponyme,E,1,,False
4,Wikispecies,Wikispecies,E,1,,False
5,Wikivoyage,Wikivoyage,E,1,,False
6,Taxonomie,Artikel Biologische Taxonomie,E,1,,False
7,Präfix,Präfixe (Deutsch),E,1,,False
8,Suffix,Suffixe (Deutsch),E,1,,False
9,VMaßeinheiten,Vorsätze für Maßeinheiten,E,1,,False


time: 49 ms


In [31]:
# all: 2h 22min
# 100K: 8min 44s
# ERROR1 2ERROR
# apply erzeugt eine Series von DataFrames
# tolist macht daraus eine Liste
# und concat macht aus der Liste von DataFrames ein Gesamt-DataFrame
wiktionary_merkmal = pd.concat(  wiktionary.swifter.apply(extract_all, plan=plan, axis=1).tolist()  )

Pandas Apply:   0%|          | 0/837593 [00:00<?, ?it/s]

ERROR1 2ERROR
time: 13h 7min 40s


In [32]:
# nachbearbeiten
wiktionary_merkmal = pak.reset_index( wiktionary_merkmal )
wiktionary_merkmal = pak.move_cols(   wiktionary_merkmal, ['section_id','section_id2','merkmal'])
wiktionary_merkmal = pak.drop_cols(   wiktionary_merkmal, ['node_link'])
wiktionary_merkmal = pak.rename_col(  wiktionary_merkmal, 'node_num',  'num')
wiktionary_merkmal = pak.rename_col(  wiktionary_merkmal, 'node_meta', 'meta')
wiktionary_merkmal = pak.rename_col(  wiktionary_merkmal, 'node_text', 'data')    

time: 8.8 s


## Kontrolle: wiktionary_merkmal

In [33]:
# Merkmale Häufigkeit
v = pak.analyse_freqs(wiktionary_merkmal, 'merkmal') 
v

,merkmal,merkmal_count,merkmal_percent,graph
0,gm,2565289,48.6,########################
1,übersicht,1229654,23.3,###########
2,def,290370,5.5,##
3,familie,181434,3.4,#
4,down,170792,3.2,#
5,kombi,153988,2.9,#
6,alt_1,147359,2.8,#
7,syn,143937,2.7,#
8,up,111771,2.1,#
9,lateral,110180,2.1,#


time: 1.54 s


In [34]:
# plan erfüllt?
geplant = set(plan.merkmal)
done    = set(v.merkmal)
nicht_erfüllt = geplant - done
print(nicht_erfüllt)
if version == '_all': 
    assert len(nicht_erfüllt) <= 1  # {'reim'} ist nicht erfüllt

{'down_part'}
time: 38.2 ms


In [35]:
# Suche festlegen
suche_debug = ['Haus',]

time: 32.8 ms


In [36]:
# Alle Merkmale einer Section
mask = wiktionary.section_id2.isin(suche_debug)
suche_section_ids = wiktionary[mask].section_id.tolist()
mask = wiktionary_merkmal.section_id.isin(suche_section_ids)
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data'], limits=[999,None])
#grid(a)
a

,merkmal,merkmal_count,merkmal_percent,data,data_count
0,kombi,416,33.8,"[ein, das, die, gehen, sein, jemanden, nach ''...","[27, 11, 6, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4..."
1,down,395,32.1,"[Hochzeitshaus, Doppelhaus, Kartenhaus, Apartm...","[3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,familie,275,22.3,"[hausschlachten, aushäusig, dreihäusig, einhäu...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,lateral,49,4.0,"[Spelunke, Bleibe, Klause, Logis, Quartier, Un...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,übersicht,38,3.1,"[mini, Haus, Häuser, 2, n, Hauses, Hause, Häus...","[7, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,up,18,1.5,"[Ort, Bau, Gebäude, Personengruppe, Personenkr...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,syn,16,1.3,"[Winde, Affenkasten, Kasten, Wohnhaus, Wohngeb...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
7,herkunft,8,0.6,"[hūs, huus, hús]","[6, 1, 1]"
8,klein,7,0.6,"[Häuschen, Häuserchen, Häusli, Häusl, Häusel, ...","[1, 1, 1, 1, 1, 1, 1]"
9,def,5,0.4,"[Gebäude, Weichtier, Gastropode]","[3, 1, 1]"


time: 260 ms


In [37]:
# Alle Merkmale einer Section
mask = wiktionary.title.isin(suche_debug)
suche_section_ids = wiktionary[mask].section_id.tolist()
mask = wiktionary_merkmal.section_id.isin(suche_section_ids)
#grid(wiktionary_merkmal, mask)
wiktionary_merkmal[mask]

,section_id,section_id2,merkmal,node_debug,node_kontext,data,num,meta
10736,21700,Haus,übersicht,Deutsch Substantiv Übersicht\n,Genus,n,NaN,NaN
10737,21700,Haus,übersicht,Deutsch Substantiv Übersicht\n,Nominativ Singular,Haus,NaN,NaN
10738,21700,Haus,übersicht,Deutsch Substantiv Übersicht\n,Nominativ Plural,Häuser,NaN,NaN
10739,21700,Haus,übersicht,Deutsch Substantiv Übersicht\n,Genitiv Singular,Hauses,NaN,NaN
10740,21700,Haus,übersicht,Deutsch Substantiv Übersicht\n,Genitiv Plural,Häuser,NaN,NaN
...,...,...,...,...,...,...,...,...
11965,21700,Haus,familie,,,haushalten,[],Adjektive
11966,21700,Haus,familie,,,hausieren,[],Adjektive
11967,21700,Haus,familie,,,hausschlachten,[],Adjektive
11968,21701,Haus_1,alt_1,,,Haues,[],


time: 123 ms


In [38]:
# Ein Merkmal in allen Sections
mask = (wiktionary_merkmal.merkmal == 'up_part')
#grid(wiktionary_merkmal, mask)
wiktionary_merkmal[mask]

,section_id,section_id2,merkmal,node_debug,node_kontext,data,num,meta
12729,22500,Wiktionary,up_part,,,Wikimedia,[1],
22187,35801,Lenzing_1,up_part,,,Vöcklabruck,[1],
22188,35801,Lenzing_1,up_part,,,Oberösterreich,[1],
27637,46800,Tag,up_part,,,Woche,[2],
27638,46800,Tag,up_part,,,Monat,[2],
...,...,...,...,...,...,...,...,...
5271313,111677500,Steuerlehre,up_part,,,Steuerwissenschaft,[1],
5272180,111693800,Bimatrix,up_part,,,Zweipersonenspiel,[1],
5274181,111731900,Lebensmittelkonzern,up_part,,,Lebensmittelbranche,[1],
5274182,111731900,Lebensmittelkonzern,up_part,,,Lebensmittelindustrie,[1],


time: 292 ms


## Müll löschen

In [39]:
# Reste von Bildern
mask1 = (wiktionary_merkmal.merkmal == 'übersicht')
mask2 = (wiktionary_merkmal.node_kontext.str.len() < 3)
mask = mask1  &  mask2
wiktionary_merkmal = pak.drop_rows(wiktionary_merkmal,mask, verbose=True)

Delete 108208 rows from 5279625
time: 3.04 s


In [40]:
# wiktionary_merkmal: Spalten löschen
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,['section_id'])

time: 1.08 s


## Speichern

In [41]:
# Speichern
if wiktionary_merkmal_filename:
    pak.dump_pickle(wiktionary_merkmal, wiktionary_merkmal_filename) 

time: 31.2 s


In [42]:
# Ausgabeverzeichnis touch
import os
os.utime(verzeichnis_save)

time: 47.8 ms


In [43]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive

time: 57.8 ms


In [44]:
# memory_consumption
pak.memory_consumption( locals() )

,name,rtype,size
0,wiktionary,"(DataFrame, Series)",3.0 GB
1,wiktionary_merkmal,"(DataFrame, Series)",2.2 GB
2,wiktionary_trash,"(DataFrame, Series)",16.1 MB
3,mask,"(Series, bool)",5.0 MB
4,mask1,"(Series, bool)",5.0 MB
5,mask2,"(Series, bool)",5.0 MB
6,__,"(DataFrame, Series)",534.9 KB
7,_37,"(DataFrame, Series)",534.9 KB
8,testzeile,"(Series, int32)",347.7 KB
9,_,"(DataFrame, Series)",199.9 KB


time: 12.6 s


In [45]:
raise Stop

Stop Time:  15:04:43
Elapsed:    17 hours, 2 mins, 26 secs


time: 61.3 ms
